In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
!pip install tf2cv tensorflow>=2.0.0

In [ ]:
from tensorflow.python.client import device_lib
dev=device_lib.list_local_devices()
dev[len(dev)-1]

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15399512821617910197
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"

In [ ]:
#global consts
IMG_SIZE='137x236'
HEIGHT = 137
WIDTH = 236
BATCH_SIZE=128
DROP_OUT=0.0
FOLD=4
TRAIN_DIR = './train/'
TRAIN_IMG_DIR=TRAIN_DIR+str(IMG_SIZE)
EPOCHS = 200
STATS_FS = (0.055029564364430086, 0.17228061284674265)
STATS_128 = (0.0692, 0.2051)
EXPERIMENT='sernext_fullsize_5fold_f4'
INPUT_DIR ='/content/drive/My Drive/kaggle/bengali/input/'
OUTPUT_DIR = '/content/drive/My Drive/kaggle/bengali/output/'
IMPORT_DIR = '/content/drive/My Drive/kaggle/bengali/nbs/'

EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'
CHKP_DIR = EXPERIMENT_DIR+'chkp/'
OUTPUT_SUBDIR = ''
DATASET_224='224x224-bengali.zip'
DATASET_128='grapheme-imgs-128x128.zip'
DATASET_FS = 'bengali-full-size.zip'
TRAIN_WITH_FOLD = 'train_with_fold.csv'
PRETRAINED = False
PRETRAINED_WEIGHTS = ''
LOG_FILE = EXPERIMENT_DIR+'log_'+EXPERIMENT+'.csv'
#INITIAL_EPOCH=40
#HEAD_WD=1e-3
#BACKBONE_WD = 1e-4

if IMG_SIZE == 128: DATASET=DATASET_128
if IMG_SIZE == 224: DATASET=DATASET_224
if IMG_SIZE == '137x236' : DATASET=DATASET_FS


#lr sheduler consts
LR_MAX = 0.001
LR_MIN = 0.00001
T_MAX = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys
sys.path.append(IMPORT_DIR+'py/models/')
sys.path.append(IMPORT_DIR+'py/utils/')
sys.path.append(IMPORT_DIR+'py/callbacks/')
sys.path.append(IMPORT_DIR+'py/opts/')

In [ ]:
#import local modules
#from mish import Mish,mish
from generators import *
from kaggle_metric import *
from cosine import CosineAnnealingScheduler
from kagglevalidation import KaggleValidationFS
#from lookahead import Lookahead
from se_resnext50 import build_se_resnext50_model_fs_wod
#from md121v2 import build_md121_v2_model
#from losses import categorical_focal_loss


#import external modules
import numpy as np 
import pandas as pd 

import os
import errno

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

In [ ]:
try:
  os.mkdir(TRAIN_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_DIR+' already exists')
    else:
        raise
try:
  os.mkdir(TRAIN_IMG_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_IMG_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(EXPERIMENT_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(EXPERIMENT_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(CHKP_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(CHKP_DIR+' already exists')
    else:
        raise


os.system('cp '+ '"'+INPUT_DIR+DATASET+'" ' + TRAIN_DIR)
os.system('cp '+ '"'+INPUT_DIR+TRAIN_WITH_FOLD+'" ' + TRAIN_DIR)

/content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/ already exists
/content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ already exists


0

In [ ]:
os.system('unzip -q '+TRAIN_DIR+DATASET+ ' -d '+ TRAIN_IMG_DIR)

0

In [ ]:
!ls train

137x236  bengali-full-size.zip	train_with_fold.csv


In [ ]:
dataset_np = pd.read_csv(TRAIN_DIR+TRAIN_WITH_FOLD).to_numpy()
dataset_np[:,0]+='.png'
dataset_m = dataset_np.shape[0]


valid_m = np.where(dataset_np[:,6]==FOLD)[0].shape[0]
train_m = dataset_m-valid_m

fold_train_inds = np.where(dataset_np[:,6] != FOLD)
fold_valid_inds = np.where(dataset_np[:,6] == FOLD)

train_np = dataset_np[fold_train_inds]
valid_np = dataset_np[fold_valid_inds]

train_df = pd.DataFrame(train_np)
valid_df = pd.DataFrame(valid_np)

train_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)
valid_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)

train_df.drop(columns=[5,6], inplace=True)
valid_df.drop(columns=[5,6], inplace=True)

In [ ]:
model =  build_se_resnext50_model_fs_wod(height=HEIGHT, width=WIDTH)

In [ ]:
cutout_datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(p=0.95,s_l=0.05,s_h=0.3,r_1=0.8,r_2=1.2,v_l=0,v_h=0))

mcm_datagen = ImageDataGenerator()

In [ ]:
columns=["root_class","vowel_class", "cons_class"]

mcm_generator1 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

mcm_generator2 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")


cutout_generator = cutout_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

Found 160672 validated image filenames.
Found 160672 validated image filenames.
Found 160672 validated image filenames.


In [ ]:
def g3_generator_fs2(gen1,gen2, gen3,h=137,w=236,st=STATS_FS):
    while True:
        p = np.random.rand()
        if p>0.8:
            batch = next(gen3)
            x_res,y_res = prep_batch_fs_shuffled(batch,st=st)
        else:
            batch1 = next(gen1)
            batch2 = next(gen2)
            q = np.random.rand()
            if q>0.3:
                x_res,y_res = mix_batches_fs_shuffled(batch1,batch2,alpha=0.1,st=st)
            else:
                x_res,y_res = cutmix_batches_fs2_shuffled(batch1,batch2,alpha=0.8,h=h,w=w,st=st)
        yield x_res,y_res

In [ ]:
g3_datagen_fs2 = g3_generator_fs2(mcm_generator1,mcm_generator2,cutout_generator,h=HEIGHT,w=WIDTH, st=STATS_FS)

In [ ]:
if 0: 
  batch_x,y = next(g3_datagen_fs2)
  import matplotlib.pyplot as plt
  fig, axs = plt.subplots(1, 7, figsize=(30, 30))
  for i in range(7):
    axs[i].imshow(batch_x[i,:,:,0], cmap='gray')
  plt.show()

In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=LR_MAX)

In [ ]:
checkpoint =  tf.train.Checkpoint(latest_epoch=tf.Variable(0), optimizer=opt, model=model)
chkp_manager =  tf.train.CheckpointManager(checkpoint, CHKP_DIR, max_to_keep=3)

In [ ]:
chkp_manager.latest_checkpoint

'/content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-50'

In [ ]:
chkp_restore_status=checkpoint.restore(chkp_manager.latest_checkpoint)
if chkp_manager.latest_checkpoint:
  #chkp_restore_status.assert_consumed()
  print("Restored from {}".format(chkp_manager.latest_checkpoint))
  print(f"Lastest epoch {checkpoint.latest_epoch.numpy()}")
else:
  print("Initializing from scratch.")

Restored from /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-50
Lastest epoch 50


In [ ]:
loss_dict={'root':    'categorical_crossentropy',
           'vowel':    'categorical_crossentropy',
           'consonant':'categorical_crossentropy'}
model.compile(optimizer=opt, loss=loss_dict, loss_weights=[4.0,1.0,1.0])

In [ ]:
initial_epoch = checkpoint.latest_epoch.numpy()

print(f'epochs trained: {initial_epoch}. start training epoch {initial_epoch+1}...')
#ca_shed = CosineAnnealingScheduler(T_max=100, eta_max=0.0001, eta_min=0.000001,verbose=1, initial_epoch=59)
#rolrplt = tf.keras.callbacks.ReduceLROnPlateau(monitor='kaggle',mode='max', patience=5, verbose=1, factor=0.5, min_lr=LR_MIN)


suffix =EXPERIMENT+\
                '_eps'+str(EPOCHS)+\
                '_iniep'+str(initial_epoch)+\
                '_sz'+str(IMG_SIZE)+\
                '_bs'+str(BATCH_SIZE)+\
                '_do'+str(DROP_OUT)+\
                '_'+model.optimizer.get_config()['name']+\
                '_IS_fold'+str(FOLD)


kv = KaggleValidationFS(valid_df,
                      batch_size=512,
                      vals_per_epoch=1,
                      train_steps=train_m//BATCH_SIZE+1,
                      logfile=EXPERIMENT_DIR+'log_kr_'+suffix+'.csv',
                      initial_epoch=initial_epoch,
                      suffix = suffix,
                      train_img_dir=TRAIN_IMG_DIR,
                      output_dir=EXPERIMENT_DIR,
                      thresholds=[0.987,0.995,0.994],
                      chkp_manager=chkp_manager,
                      in_shape=[HEIGHT,WIDTH],
                      st=STATS_FS)

tf.keras.backend.set_value(model.optimizer.lr, 0.0001)
cbs = [kv]

history=model.fit(g3_datagen_fs2,
                  epochs=EPOCHS,
                  steps_per_epoch=train_m//BATCH_SIZE+1,
                  callbacks=cbs,
                  verbose=2)

epochs trained: 66. start training epoch 67...
Train for 1256 steps
Epoch 1/200

batch 1255: validation...



[0:20:51] - kaggle:0.9855 - root:0.9826 -vowel:0.9895 - cons: 0.9873

saved checkpoint for epoch 67: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-67

1256/1256 - 1254s - loss: 3.1042 - root_loss: 0.6279 - vowel_loss: 0.3644 - consonant_loss: 0.2284
Epoch 2/200

batch 1255: validation...



[0:41:46] - kaggle:0.9847 - root:0.9814 -vowel:0.9896 - cons: 0.9864

saved checkpoint for epoch 68: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-68

1256/1256 - 1255s - loss: 3.0811 - root_loss: 0.6213 - vowel_loss: 0.3659 - consonant_loss: 0.2301
Epoch 3/200

batch 1255: validation...



[1:02:39] - kaggle:0.9852 - root:0.9825 -vowel:0.9899 - cons: 0.9859

saved checkpoint for epoch 69: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-69

1256/1256 - 1253s - loss: 3.2369 - root_loss: 0.6535 - vowel_loss: 0.3836 - consonant_loss: 0.2393
Epoch 4/200

batch 1255: validation...



[1:23:30] - kaggle:0.9859 - root:0.9835 -vowel:0.9896 - cons: 0.9869

saved checkpoint for epoch 70: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-70

1256/1256 - 1251s - loss: 2.8439 - root_loss: 0.5720 - vowel_loss: 0.3416 - consonant_loss: 0.2144
Epoch 5/200

batch 1255: validation...


...


[2:26:04] - kaggle:0.9840 - root:0.9813 -vowel:0.9899 - cons: 0.9835

saved checkpoint for epoch 73: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-73

1256/1256 - 1250s - loss: 2.9030 - root_loss: 0.5842 - vowel_loss: 0.3474 - consonant_loss: 0.2188
Epoch 8/200

batch 1255: validation...



[2:46:58] - kaggle:0.9855 - root:0.9831 -vowel:0.9909 - cons: 0.9848

saved checkpoint for epoch 74: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-74

1256/1256 - 1255s - loss: 2.9277 - root_loss: 0.5878 - vowel_loss: 0.3545 - consonant_loss: 0.2221
Epoch 9/200

batch 1255: validation...



[3:07:50] - kaggle:0.9851 - root:0.9826 -vowel:0.9905 - cons: 0.9847

saved checkpoint for epoch 75: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-75

1256/1256 - 1251s - loss: 2.9672 - root_loss: 0.5962 - vowel_loss: 0.3574 - consonant_loss: 0.2251
Epoch 10/200

batch 1255: validation...



[3:28:40] - kaggle:0.9854 - root:0.9826 -vowel:0.9903 - cons: 0.9863

saved checkpoint for epoch 76: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-76

1256/1256 - 1250s - loss: 3.1097 - root_loss: 0.6253 - vowel_loss: 0.3755 - consonant_loss: 0.2329
Epoch 11/200

batch 1255: validation...



[3:49:33] - kaggle:0.9862 - root:0.9832 -vowel:0.9907 - cons: 0.9876

saved checkpoint for epoch 77: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f4/chkp/ckpt-77

1256/1256 - 1252s - loss: 2.9078 - root_loss: 0.5838 - vowel_loss: 0.3519 - consonant_loss: 0.2208
Epoch 12/200


In [ ]:
model.optimizer.get_config()